In [1]:
import pandas as pd
import numpy as np
from indicators import calc_rsi 
from indicators import calc_macd

In [2]:

ticker = "Nvidia"
df = pd.read_csv(f'./data_set/{ticker}.csv').head(1000) 
bull = []
bear = []
horizontal = []
result = [] 
for i in range(len(df)):
    if (df.close.iloc[i]) < (df.open.iloc[i]* 0.9980):
        bear.append(i)
        result.append("bear")
    elif df.close.iloc[i] > (df.open.iloc[i]* 1.002):
        bull.append(i) 
        result.append("bull")
    else: 
        horizontal.append(i)
        result.append("horizontal")

print("Bull days are:", bull,len(bull)) 
print("Bear days are:", bear,len(bear))
print("Horizontal days are:", horizontal,len(horizontal))
print("Combined result is: ",result)
print("The lenght of data size is ",len(result))
np.save("result.npy",result)

Bull days are: [1, 4, 7, 9, 11, 14, 15, 17, 20, 23, 26, 30, 32, 34, 36, 40, 41, 42, 47, 53, 56, 58, 60, 62, 66, 67, 72, 74, 75, 77, 79, 81, 82, 84, 86, 89, 90, 99, 101, 102, 103, 105, 107, 108, 111, 112, 113, 114, 115, 116, 117, 119, 120, 121, 122, 123, 125, 129, 131, 132, 143, 146, 149, 150, 152, 156, 158, 159, 160, 161, 164, 165, 166, 169, 172, 173, 176, 177, 179, 180, 183, 184, 185, 190, 192, 197, 198, 199, 200, 201, 202, 203, 209, 213, 214, 215, 217, 219, 224, 228, 232, 236, 237, 245, 246, 249, 252, 253, 254, 257, 259, 260, 262, 263, 264, 265, 267, 269, 271, 273, 275, 278, 280, 281, 283, 286, 288, 289, 294, 295, 298, 299, 300, 302, 307, 310, 312, 313, 316, 321, 325, 326, 330, 331, 332, 333, 336, 337, 338, 341, 343, 344, 345, 347, 351, 352, 353, 356, 357, 358, 359, 360, 363, 365, 370, 377, 379, 380, 384, 387, 389, 390, 391, 398, 400, 402, 404, 406, 407, 408, 411, 414, 419, 420, 422, 423, 424, 426, 432, 433, 434, 435, 438, 439, 440, 442, 443, 444, 448, 450, 451, 452, 453, 454, 455, 4

In [3]:
# calculate the probablity that when a day is bull next day is also bull
# calculate the probablity that when a day is bull next day is horizontal
# calculate the probablity that when a day is bull next day is bear
bull_2_bull = 0
bull_2_horizontal=0
bull_2_bear = 0

bear_2_bull = 0
bear_2_horizontal = 0
bear_2_bear = 0

horizontal_2_bull = 0
horizontal_2_horizontal = 0
horizontal_2_bear = 0

In [4]:
prob_bull = np.zeros(3,dtype=int)
prob_bear = np.zeros(3,dtype=int)
prob_horizontal = np.zeros(3,dtype=int)

for i in range(len(result)-1):
    day = result[i]
    tomorrow = result[i+1]
    if day == 'bull':
        if tomorrow == 'bull':    
            prob_bull[0] += 1
        elif tomorrow == 'bear':
            prob_bull[1] += 1
        else: prob_bull[2] +=1
    elif day == 'bear':
        if tomorrow == 'bull':
            prob_bear[0] += 1   
        elif tomorrow == 'bear':
            prob_bear[1] += 1
        else: prob_bear[2] +=1
    else:
        if tomorrow == 'bull':
            prob_horizontal[0] += 1   
        elif tomorrow == 'bear':
            prob_horizontal[1] += 1
        else: prob_horizontal[2] +=1

print(prob_bull)
print(prob_bear)
print(prob_horizontal)
    


[225 222  31]
[223 197  32]
[30 32  7]


the matrix will denote that 

0,0 bull bull
0,1 bull horizontal
0,2 bull bear

1,0 horizontal bull
1,1 horizontal horizontal
1,2 horizontal bear

2,0 bear bull
2,1 bear horizontal
2,2 bear bear 


In [5]:
bull_c = np.sum(prob_bull)
bear_c = np.sum(prob_bear)
horizontal_c = np.sum(prob_horizontal)

# find the probablities
bull_bull = prob_bull[0]/bull_c
bull_bear = prob_bull[1]/bull_c
bull_horizontal = prob_bull[2]/bull_c

bear_bull = prob_bear[0]/bear_c
bear_bear = prob_bear[1]/bear_c
bear_horizontal = prob_bear[2]/bear_c

horizontal_bull = prob_horizontal[0]/horizontal_c
horizontal_bear = prob_horizontal[1]/horizontal_c
horizontal_horizontal = prob_horizontal[2]/horizontal_c

matrix = np.array([
                    [bull_bull,bull_horizontal,bull_bear],
                    [horizontal_bull,horizontal_horizontal,horizontal_bear],
                    [bear_bull,bear_horizontal,bear_bear]])
#print(matrix)
print(df.loc[df.index.max()])  # Mevcut en büyük index değerini gösterir
np.save("matrix.npy",matrix)



date      2024-10-04 00:00:00-04:00
open                     124.919903
high                     125.019886
low                      121.810403
close                    124.899902
volume                    243678100
Name: 999, dtype: object


Lets see what markov chain tells us for the next day 

We trade at the days what the indicators allows as (denoted as 1)